In [1]:
import numpy as np
import matplotlib.pyplot as plt

import qnmfits

sim = qnmfits.SXS(ID=305, zero_time=(2,2))

Skipping download from 'https://data.black-holes.org/catalog.json' because local file is newer
Found the following files to load from the SXS catalog:
    SXS:BBH:0305v5/Lev6/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:0305v5/Lev6/rhOverM_Asymptotic_GeometricUnits_CoM.h5


In [3]:
spherical_modes = [(2,2), (2,1), (2,0), (3,3), (3,2), (3,1), (3,0)]

modes, _, _ = qnmfits.greedy_fit(
    sim.times, 
    sim.h, 
    Ftarget=0.001, 
    Nmax=10, 
    Mf=sim.Mf, 
    chif=sim.chif_mag, 
    t0=30, 
    spherical_modes=spherical_modes
)

print(modes)

[(2, 2, 0, 1), (3, 3, 0, 1), (2, 1, 0, 1), (3, 2, 0, 1)]
